In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data import Dataset, DataLoader

import os
import cv2
from PIL import Image
from tqdm.notebook import tqdm
from tqdm.contrib.telegram import tqdm as tgdm_tg
from prettytable import PrettyTable
import random
import numpy as np
import gc
import scipy.io as sio
import pandas as pd
import time

from sklearn.metrics import f1_score, accuracy_score, matthews_corrcoef

from torchvision import datasets, transforms as T

from transformers import AutoTokenizer, AutoModel

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from pynvml import nvmlInit, nvmlDeviceGetHandleByIndex, nvmlDeviceGetMemoryInfo

In [2]:
!nvidia-smi

Sun Oct 23 00:28:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100 Graphics D...  On   | 00000000:B7:00.0 Off |                    0 |
| N/A   36C    P0    74W / 400W |   6498MiB / 81252MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
goods = pd.read_feather("preprocess_all_columns-lem.feather")

In [5]:
n_classes = goods["Код КПГЗ"].nunique()

In [6]:
class BertCLS(nn.Module):
    def __init__(self, model, n_classes):
        super(BertCLS, self).__init__()
        self.model = model
        self.fc = nn.Linear(768, n_classes)

    def forward(self, batch):
        return self.fc(self.model(**batch).pooler_output)

In [7]:
bert_cls = BertCLS(model, n_classes)

In [8]:
n_classes

5307

In [9]:
goods

,ID СТЕ,Название СТЕ,Категория,Код КПГЗ,Характеристики,Название_СТЕ_source,Характеристики_source,target
0,1153097,мяч футбольный mikasa regateador5-g,Мячи футбольные,01.08.01.13.01,"5 усл. ед, mikasa, regateador5-r, белый, любит...",мяч футбольный MIKASA REGATEADOR5-G,"[{""Name"":""Модель"",""Id"":283795036,""Value"":""REGA...",2918
1,1153130,мяч волейбольный gala pro-line 10 fivb,Мячи волейбольные,01.08.01.14.03.01,"5, gala, pro-line 10 fivb, белый, профессионал...",мяч волейбольный Gala Pro-Line 10 FIVB,"[{""Name"":""Марка"",""Id"":284249992,""Value"":""Gala""...",2927
2,1153163,мяч волейбольный mikasa mva380k-obl,Мячи волейбольные,01.08.01.14.03.01,"5, mikasa, mva380k, оранжевый, синт, тренровочный",мяч волейбольный Mikasa MVA380K-OBL,"[{""Name"":""Марка"",""Id"":284249802,""Value"":""MIKAS...",2927
3,1153174,мяч волейбольный wilson super soft play,Мячи волейбольные,01.08.01.14.03.01,"5, super soft play, wilson, белый, любительски...",мяч волейбольный Wilson Super Soft Play,"[{""Name"":""Марка"",""Id"":284246959,""Value"":""WILSO...",2927
4,1159300,gutrend комплект расходный материал для fun 11...,"Расходные материалы, комплектующие для прочего...",01.20.10.99,"120 г, 170 мм, 220 мм, 50 мм, 6 шт, hepa фильт...",Gutrend комплект расходных материалов для FUN ...,"[{""Name"":""Тип"",""Id"":284280400,""Value"":""Расходн...",4551
...,...,...,...,...,...,...,...,...
354225,35228990,шина 23x10.50-12 107a8 starco as loader tl,"Шины для грузовых автомобилей и спецтехники, п...",01.09.08.10.05.02,"10 кг, 12 дюйм, 580 мм, 8 pr, starco, бескамер...",Шина 23x10.50-12 107A8 Starco AS LOADER TL,"[{""Name"":""Бескамерные"",""Id"":369460372,""Value"":...",3077
354226,35229086,кнопка включение рабочий освещение для минипог...,Запчасти к погрузчикам,01.09.08.03,"12 в, 650, 90 г, avant, включение рабочий осве...",Кнопка включения рабочего освещения для минипо...,"[{""Name"":""Артикул запчасти"",""Id"":369463362,""Va...",3067
354227,35231655,обои флизелиновый под покраска nc antivandal а...,Обои,01.11.03.11.08,"106 смотреть, 25 м, 350 г, 4 шт, 4010-16, anti...","Обои флизелиновые под покраску NC Antivandal, ...","[{""Name"":""Описание"",""Id"":369399594,""Value"":""со...",3432
354228,35231695,мусорный ведро kimberly-clark aquarius белый п...,Контейнеры и другие емкости для мусора пластма...,01.20.03.03.06,"2 шт, 29.00000 смотреть, 4,76 кг, 43.00000 смо...",Мусорное ведро Kimberly-Clark Aquarius белое п...,"[{""Name"":""Вид емкости"",""Id"":369380812,""Value"":...",4345


In [10]:
bert_cls.load_state_dict(torch.load("./models-all-classes-lem/BertCLS_epoch_5.pth", map_location=torch.device("cpu")))

<All keys matched successfully>

In [11]:
class ClassificationDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __getitem__(self, idx):
        text = self.data[idx]
        return text

    def __len__(self):
        return len(self.data)

def collate_fn(batch):
    model_input = []
    for text in batch:
        model_input.append(text)

    tok = tokenizer(model_input, padding=True,
                    max_length=200, truncation=True,
                    return_tensors='pt')
    return tok

In [12]:
def get_loader(dataset, shuffle, batch_size):
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=False,
        num_workers=0,
        collate_fn=collate_fn
    )
    return loader

In [13]:
batch_size = 200
train_dataset = ClassificationDataset(goods["Название СТЕ"].values)
train_loader = get_loader(train_dataset, shuffle=False, batch_size=batch_size)

In [14]:
bert_cls = bert_cls.to(device)

In [15]:
def test(model, loader, device):
    pred = []
    model.eval()
    with torch.no_grad():
        pbar = tgdm_tg(loader, token="5258964872:AAGPTJDWI2QBOqe_5jqlNqKr-fZf_xwhcEs", chat_id="661328720")
        for batch_idx, data in enumerate(pbar):
            data = data.to(device)
            embeddings = model.model(**data).pooler_output.detach().cpu().numpy().astype(np.float16)
            pred.extend(embeddings)
    return pred

In [16]:
logits = test(bert_cls, train_loader, device)

/home/jovyan/minerals/gerasimov/resnet-training/max_resnet_venv/lib/python3.7/site-packages/tqdm/contrib/telegram.py:38: TqdmWarning: Creation rate limit: try increasing `mininterval`.
  self.message_id


  0%|          | 0/1772 [00:00<?, ?it/s]

/home/jovyan/minerals/gerasimov/resnet-training/max_resnet_venv/lib/python3.7/site-packages/tqdm/contrib/telegram.py:66: TqdmWarning: Creation rate limit: try increasing `mininterval`.
  message_id = self.message_id


In [17]:
logits[0].shape

(768,)

In [18]:
len(logits)

354230

In [19]:
df = pd.DataFrame(logits)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354230 entries, 0 to 354229
Columns: 768 entries, 0 to 767
dtypes: float16(768)
memory usage: 518.9 MB


In [21]:
final_df = pd.concat([goods, df], axis=1)

In [22]:
final_df

,ID СТЕ,Название СТЕ,Категория,Код КПГЗ,Характеристики,Название_СТЕ_source,Характеристики_source,target,0,1,...,758,759,760,761,762,763,764,765,766,767
0,1153097,мяч футбольный mikasa regateador5-g,Мячи футбольные,01.08.01.13.01,"5 усл. ед, mikasa, regateador5-r, белый, любит...",мяч футбольный MIKASA REGATEADOR5-G,"[{""Name"":""Модель"",""Id"":283795036,""Value"":""REGA...",2918,-0.861328,-0.281494,...,-0.979004,0.903320,-0.390625,0.633789,-0.330322,-0.989258,-0.988770,-0.189331,-0.998535,0.505859
1,1153130,мяч волейбольный gala pro-line 10 fivb,Мячи волейбольные,01.08.01.14.03.01,"5, gala, pro-line 10 fivb, белый, профессионал...",мяч волейбольный Gala Pro-Line 10 FIVB,"[{""Name"":""Марка"",""Id"":284249992,""Value"":""Gala""...",2927,0.953125,0.224609,...,-0.966309,0.876953,-0.990723,-0.923828,0.994629,0.339111,-0.653320,0.997070,-0.637207,0.967285
2,1153163,мяч волейбольный mikasa mva380k-obl,Мячи волейбольные,01.08.01.14.03.01,"5, mikasa, mva380k, оранжевый, синт, тренровочный",мяч волейбольный Mikasa MVA380K-OBL,"[{""Name"":""Марка"",""Id"":284249802,""Value"":""MIKAS...",2927,0.970703,-0.131470,...,-0.961914,0.951660,-0.987793,-0.901855,0.983887,0.111023,-0.740234,0.995605,-0.937012,0.932617
3,1153174,мяч волейбольный wilson super soft play,Мячи волейбольные,01.08.01.14.03.01,"5, super soft play, wilson, белый, любительски...",мяч волейбольный Wilson Super Soft Play,"[{""Name"":""Марка"",""Id"":284246959,""Value"":""WILSO...",2927,0.959473,0.237427,...,-0.983887,0.894043,-0.991699,-0.957031,0.984375,0.204590,-0.837891,0.989746,-0.652344,0.960449
4,1159300,gutrend комплект расходный материал для fun 11...,"Расходные материалы, комплектующие для прочего...",01.20.10.99,"120 г, 170 мм, 220 мм, 50 мм, 6 шт, hepa фильт...",Gutrend комплект расходных материалов для FUN ...,"[{""Name"":""Тип"",""Id"":284280400,""Value"":""Расходн...",4551,0.971191,-0.447266,...,0.998535,0.986816,0.307373,0.193237,0.541992,-0.900391,-0.449463,0.994629,0.350098,0.895508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354225,35228990,шина 23x10.50-12 107a8 starco as loader tl,"Шины для грузовых автомобилей и спецтехники, п...",01.09.08.10.05.02,"10 кг, 12 дюйм, 580 мм, 8 pr, starco, бескамер...",Шина 23x10.50-12 107A8 Starco AS LOADER TL,"[{""Name"":""Бескамерные"",""Id"":369460372,""Value"":...",3077,0.988770,0.800293,...,-0.864258,0.978516,-0.859375,0.079102,-0.295898,-0.991211,-0.958496,-0.539062,-0.107239,0.979492
354226,35229086,кнопка включение рабочий освещение для минипог...,Запчасти к погрузчикам,01.09.08.03,"12 в, 650, 90 г, avant, включение рабочий осве...",Кнопка включения рабочего освещения для минипо...,"[{""Name"":""Артикул запчасти"",""Id"":369463362,""Va...",3067,-0.336182,0.863281,...,0.955566,0.995605,-0.843750,-0.852539,-0.328857,0.979004,-0.939453,0.993652,-0.303467,0.977051
354227,35231655,обои флизелиновый под покраска nc antivandal а...,Обои,01.11.03.11.08,"106 смотреть, 25 м, 350 г, 4 шт, 4010-16, anti...","Обои флизелиновые под покраску NC Antivandal, ...","[{""Name"":""Описание"",""Id"":369399594,""Value"":""со...",3432,-0.998535,-0.958496,...,0.371094,0.993652,-0.882812,-0.971680,-0.995605,-0.997070,-0.379639,-0.892090,0.975586,-0.907715
354228,35231695,мусорный ведро kimberly-clark aquarius белый п...,Контейнеры и другие емкости для мусора пластма...,01.20.03.03.06,"2 шт, 29.00000 смотреть, 4,76 кг, 43.00000 смо...",Мусорное ведро Kimberly-Clark Aquarius белое п...,"[{""Name"":""Вид емкости"",""Id"":369380812,""Value"":...",4345,0.997559,0.981445,...,-0.992676,0.874023,0.955566,-1.000000,0.995605,-0.999512,0.228882,0.995605,0.226685,0.920898


In [23]:
final_df.columns = [str(i) for i in final_df.columns]

In [25]:
final_df.to_feather("LABSE-5307-epoch-5-lem.feather")

In [ ]:
df.to_feather("bert-tiny-1500-embedings.feather")

In [ ]:
temp.to_feather("bert-tiny-1500-info.feather")

In [ ]:
final_df

In [33]:
import pymorphy2
import nltk
from string import punctuation

# nltk.download('punkt', download_dir="/home/jovyan/nltk_data")

punctuation = set(punctuation)
morph = pymorphy2.MorphAnalyzer()

In [34]:
def clear_text(text):
    tokens = nltk.word_tokenize(text, language="ru")
    tokens = [morph.parse(i)[0].normal_form for i in tokens if i not in punctuation]
    return " ".join(tokens)

In [27]:
import faiss

In [28]:
embeddings = df.values.astype(np.float32)

In [29]:
embeddings.shape

(354230, 768)

In [30]:
embeddings = np.ascontiguousarray(embeddings)

In [31]:
d = 768 # длина эмбеддинга
index = faiss.IndexFlatIP(d)
index.add(embeddings) # сами эмбеддинги, нампай массив shape = (n_samples, d)

In [32]:
from pyjarowinkler import distance

def string_dist(str1, str2):
    return distance.get_jaro_distance(str1, str2,
                                      winkler=False,
                                      winkler_ajustment=False,
                                      scaling=0.2)

In [49]:
example_txt = clear_text("игрушки".lower().strip())
example = tokenizer(example_txt, padding=True,
                    max_length=200, truncation=True,
                    return_tensors='pt')

In [50]:
with torch.no_grad():
    bert_cls.eval()
    xq = bert_cls.model(**example.to(device)).pooler_output.detach().cpu().numpy()

In [51]:
k = 50
D, I = index.search(xq, k) # xq shape = (1, d)

In [52]:
abc = goods.reset_index(drop=True).iloc[I[0]]

In [53]:
abc["dist"] = abc["Название СТЕ"].apply(lambda x: string_dist(example_txt, x))

In [54]:
abc.sort_values(by="dist", ascending=False)

,ID СТЕ,Название СТЕ,Категория,Код КПГЗ,Характеристики,Название_СТЕ_source,Характеристики_source,target,dist
82478,18820348,игрушка утюжок,Игрушки прочие,01.20.09.01.99,"2 шт, 3-6 год, игрушка утюжок, пластмасса, раз...",Игрушка . Утюжок,"[{""Name"":""Вид товара"",""Id"":-619485205,""Value"":...",4542,0.833333
157380,23611284,игрушка ящерица,Игрушки прочие,01.20.09.01.99,"100 45 мм, 12 г, 5 г ^год, резина",Игрушка Ящерица,"[{""Name"":""Материал"",""Id"":-280551228,""Value"":""р...",4542,0.822222
122098,20930364,игрушка заводной,Игрушки прочие,01.20.09.01.99,"30 г, 90 мм, заводной, пакет, пластик, развити...",Игрушка заводная,"[{""Name"":""Вид упаковки"",""Id"":-485418633,""Value...",4542,0.812500
121981,20923990,игрушка заводной,Игрушки прочие,01.20.09.01.99,"11 г, 6 мм, желтый\/черный\/белый, заводной, и...",Игрушка заводная,"[{""Name"":""Из чего сделана игрушка (состав)"",""I...",4542,0.812500
122085,20929924,игрушка заводной,Игрушки прочие,01.20.09.01.99,"13 3 10 смотреть, 15 г, 3 г ^год, пластик, чёрный",Игрушка заводная,"[{""Name"":""Материал:"",""Id"":-485415560,""Value"":""...",4542,0.812500
121237,20881191,игрушка заводной,Игрушки прочие,01.20.09.01.99,"11 г, 5 мм, заводной, пакет, пластик",Игрушка заводная,"[{""Name"":""Вид упаковки"",""Id"":-487374431,""Value...",4542,0.812500
122088,20930033,игрушка заводной,Игрушки прочие,01.20.09.01.99,"26 г, 3 г ^год, 8 5 6 смотреть, заводной, пластик",Игрушка заводная,"[{""Name"":""Материал"",""Id"":-485382669,""Value"":""П...",4542,0.812500
121760,20911884,игрушка стрелялка,Игрушки прочие,01.20.09.01.99,"3 г ^год, 7 г, 70 15 100 мм, пакет, пластик",Игрушка Стрелялка,"[{""Name"":""Вид упаковки"",""Id"":-486209973,""Value...",4542,0.803922
116574,20636339,игрушка музыкальный,Игрушки прочие,01.20.09.01.99,"170 мм, 2 г ^год, 76 г, бубна, дерево металл, ...",Игрушка музыкальная,"[{""Name"":""Вид игрушки"",""Id"":-501131041,""Value""...",4542,0.789474
252145,34534564,игрушка `` швабра '',Игрушки прочие,01.20.09.01.99,"10x9x60 смотреть, 130 г, желтый\/синий\/розовы...","Игрушка ""Швабра""","[{""Name"":""Материал"",""Id"":355218705,""Value"":""пл...",4542,0.783333
